In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d
from scipy.spatial.transform import Rotation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
attr_map = np.load("source_data_22_9_8_2028/attri_map_22_9_8_2028.npy")
pcl = np.load("source_data_22_9_8_2028/pcl_22_9_8_2028.npy")
box = np.load("source_data_22_7_21_1140/base_det_boxes_22_7_21_1140.npy")
label = np.load("source_data_22_7_21_1127/base_det_labels_22_7_21_1127.npy")

直接提取bbox

In [3]:
from copy import deepcopy


pts = open3d.geometry.PointCloud()
pts.points = open3d.utility.Vector3dVector(pcl[:, :3])
box = box[0,:]
#rot_mat是旋转矩阵，说明之前boundingbox是不用转的？
rot_mat = Rotation.from_rotvec([0, 0, box[6]]).as_matrix()
#转的是框，
bb = open3d.geometry.OrientedBoundingBox(box[:3], rot_mat, box[3:6])
#提取中心坐标
[center_x, center_y, center_z] = bb.get_center()
#提取顶点
verterx_set = np.asarray(bb.get_box_points)
heading = box[6]
heading = -heading
pts1 = deepcopy(pts)
R = pts.get_rotation_matrix_from_xyz(( 0, 0, heading))
pts1.rotate(R)


PointCloud with 28434 points.

可视化，看一下现在转过来没

In [4]:
vis = open3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().point_size = 4.0
vis.get_render_option().background_color = np.ones(3) * 0.25
axis_pcd = open3d.geometry.TriangleMesh.create_coordinate_frame(
            size=1.0, origin=[0, 0, 0])
vis.add_geometry(axis_pcd)

vis.add_geometry(pts1)
rot_mat1 = Rotation.from_rotvec([0, 0, 0]).as_matrix()
bb1 = open3d.geometry.OrientedBoundingBox(box[:3],rot_mat1, box[3:6])
vis.add_geometry(bb)
vis.run()
vis.destroy_window()

提取bb1中的顶点坐标，根据对齐后的顶点提取索引

In [5]:
verterx_set1 = np.asarray(bb1.get_box_points())
print(verterx_set1.shape)
max_x = np.max(verterx_set1[:,0])
max_y = np.max(verterx_set1[:,1])
max_z = np.max(verterx_set1[:,2])
min_x = np.min(verterx_set1[:,0])
min_y = np.min(verterx_set1[:,1])
min_z = np.min(verterx_set1[:,2])

(8, 3)


In [6]:
def crop_filter(cloud, min_x, max_x, min_y, max_y, min_z, max_z):
    """
    获取指定区域的点
    :param cloud: 输入点云
    :param min_x: 指定方框的x方向最小值
    :param max_x: 指定方框的x方向最大值
    :param min_y: 指定方框的y方向最小值
    :param max_y: 指定方框的y方向最大值
    :param min_z: 指定方框的z方向最小值
    :param max_z: 指定方框的z方向最大值
    :return: 方框内的点云，方框外的点云
    """
    points = np.asarray(cloud.points)

    ind = np.where((points[:, 0] >= min_x) & (points[:, 0] <= max_x) &
                   (points[:, 1] >= min_y) & (points[:, 1] <= max_y) &
                   (points[:, 2] >= min_z) & (points[:, 2] <= max_z))[0]

    inlier_cloud = cloud.select_by_index(ind)
    #outlier_cloud = cloud.select_by_index(ind, invert=True)
    return inlier_cloud, ind

寄，先放一放吧

In [7]:
crop_pts, indecies =crop_filter(pts1, min_x, max_x, min_y, max_y, min_z, max_z) 
#open3d.visualization.draw_geometries([crop_pts])
print(indecies.shape)

(0,)


In [2]:
import numpy as np
slope_box = np.load("slope_5/box_09_22_1430.npy")
slope_box.shape

(17, 7)

In [ ]:
from scipy.spatial.transform import Rotation
import open3d
label = [1 ,1 ,1 ,1, 3, 1, 3, 3, 3, 3, 2, 2, 3, 3, 1, 1, 1]
label_color = [[1,0,0], [0,1,0], [0,1,1]]
detections, data = slope_box.shape
for i in range(detections):
     box1 = box[i]
     label_bbox= label[i]
     color_bbox = color[label_bbox-1]
     bbox = open3d.geometry.OrientedBoundingBox(box1[:3], rot_mat, box1[3:6])
     bbox.color = (color_bbox[:])
     #bbox.color = (1.0, 0.0, 1.0)
     


In [3]:
pts = open3d.geometry.PointCloud()
pts.points = open3d.utility.Vector3dVector(pcl[:, :3])
open3d.visualization.draw_geometries([pts])

In [6]:
from utils import cropped_pcd
slope = np.load("slope_data/pcl_10.npy")
slope_box = np.load("slope_data/box_10.npy")
slope_pts = cropped_pcd(slope, slope_box[0, :])
pts = open3d.geometry.PointCloud()
pts.points = open3d.utility.Vector3dVector(slope_pts[:, :3])
open3d.visualization.draw_geometries([pts])
